# Tutorial: ECT for example dataset

<div>
<img src="images/graphECTgif.gif" width="800" align='center'/>
</div>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

# Only for testing purposes
from sys import path
path.append( "../../ect/ect" )
from ect import ECT, EmbeddedGraph


We are using a subset of the leaf shape dataset from [(Li et al., 2018)](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2018.00553/full) as an example in this tutorial. Samples in the dataset are coordinates $(x,y)\in\mathbb{R}^2$ that form outlines of leaves. The data is pre-centered and scaled to account for any sampling resolution differences between leaf samples.
In the published version of the dataset, these coordinates are not ordered, however in order to represent the leaf outlines as a graph, we require the coordinates be ordered such that the outline of each leaf can be traced out by the graph representation.
To address the lack of ordering of the coordinates, we use the dataset version from [(Wiley, 2023)](https://github.com/willeyna/ECT\_of\_leaves), in which they use $2$-nearest neighbor graphs to order all coordinates so as to trace the outline of each leaf.

Using coordinates for each leaf, ordered such that connecting subsequent coordinates in forms the leaf outline, we build a graph representation of each leaf sample. Each of the coordinates of the leaf are represented as vertices in the graph, with edges between vertices that are adjacent in the leaf outline and the graph embedding defined by the coordinates $(x,y)\in\mathbb{R}^2$ for each vertex. Here is an example of the graph representation for this dataset from the Cotton class, zoomed in (right, in red) to show graph structure and the subgraph highlighted in red on the full outline graph.

<div>
<img src="images/Cotton_full.png" width="400" align='center'/>
<img src="images/Cotton_zoom.png" width="400" align='center'/>
</div>

# Load in the leaf dataset
- This is a subset of the full dataset provided in [(Li et al., 2018)](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2018.00553/full), with two classes (Cotton and Ivy) selected, each containing $30$ leaf outline samples.
- The ECT of each leaf sample is computed and saved as a .npy file in a directory structure matching the original dataset.


### Here is an example loading a single leaf sample, computing the ECT, and saving the output matrix.

First define the filename of the sample to be loaded and use `np.load()` to load in the numpy file.

In [ ]:
input_filedir = 'example_data/outline_input/Cotton/1a_868a.npy'
output_filedir = 'example_data/ect_output/Cotton/1a_868a.npy'
leaf = np.load(input_filedir)

Next, we initalize a graph object for the leaf sample.

In [ ]:
G = EmbeddedGraph()

Use coordinates of the leaf outline points to add vertices (and embedding coordinates) to the graph object.

In [ ]:
valuesX = leaf[:,0]
valuesY = leaf[:,1]
for i in range(np.shape(leaf)[0]):
    G.add_node(i,valuesX[i],valuesY[i])

Add edges to the graph according to the outline order. The samples of this dataset all have coordinates listed in order of the outline of the leaf. Note that for other datasets where this isn't the case, a different method must be used to define the graph structure.

In [ ]:
for i in range(np.shape(leaf)[0]-1):
    G.add_edge(i, i+1)
G.add_edge(0,np.shape(leaf)[0]-1)

G.plot()

Now, we initialize the ECT object using $32$ directions and $48$ thresholds.

In [ ]:
num_dirs = 32
num_thresh = 48

myect = ECT(num_dirs = num_dirs, num_thresh = num_thresh)
print(f'Directions chosen are: {myect.thetas}')

Set the bounding radius to be the global dataset bounding radius, so we can use the same bounding radius for all samples of the dataset.
<div>
<img src="images/boundingbox.png" width="400"/>
</div>

**[TO DO:]** compute the dataset global radius within ect package functionality


In [ ]:
global_bound_radius = 2.9092515639765497 
myect.set_bounding_radius(global_bound_radius)
print(f'Set radius is: {myect.bound_radius}')

Now the thresholds are set.

In [ ]:
print(f'Thresholds chosen are: {myect.threshes}')

Next, we calculate the ECT using the directions and thresholds specified by myect object.

In [ ]:
print(myect.calculateECT(G))

Now we can access the saved ECT matrix and optionally plot the ECT image or save the matrix as a numpy file.

In [ ]:
# The saved ECT matrix
M = myect.get_ECT()
print(f'M has shape: {M.shape}')

myect.plot('ECT')

# save the ECT matrix as a numpy file
Path(os.path.dirname(output_filedir)).mkdir(parents=True, exist_ok=True)
print(f'Saving the ECT of sample as: {output_filedir} ...')
np.save(output_filedir, M)

### Now perform the same  process on all of the samples of the dataset, computing the ECT, and saving the output matrix.

In [ ]:
mypath = 'example_data/outline_input/'
mypath_output = 'example_data/ect_output/'


# loop through file system
classes=[]
class_count = 0
for path, subdirs, files in os.walk(mypath):
    classes.extend(subdirs)
    files = [f for f in files if not f[0] == '.']
    subdirs[:] = [d for d in subdirs if (d[0] != '.')]
    print('Computing ECT of files in ', path, '...')
    print("There are ",len(files), ' samples to load in this directory.')
    
    for name in files:
        input_filedir = os.path.join(path, name)
        leaf = np.load(input_filedir)
        
        # Define a graph object for the leaf
        G = EmbeddedGraph()

        # Use coordinates of the leaf outline points to add vertices (and embedding coordinates) to the graph object
        valuesX = leaf[:,0]
        valuesY = leaf[:,1]
        for i in range(np.shape(leaf)[0]):
            G.add_node(i,valuesX[i],valuesY[i])

        # Add edges to the graph according to the outline order
        for i in range(np.shape(leaf)[0]-1):
            G.add_edge(i, i+1)
        G.add_edge(0,np.shape(leaf)[0]-1)
        
        # initialize the ECT object using 32 directions and 48 thresholds
        myect = ECT(num_dirs = num_dirs, num_thresh = num_thresh)

        # Set the bounding radius to be the global dataset bounding radius
        # TO DO: compute the dataset global radius within ect package
        global_bound_radius = 2.9092515639765497
        myect.set_bounding_radius(global_bound_radius)

        myect.calculateECT(G)

        # The saved ECT matrix
        M = myect.get_ECT()


        # save the ECT matrix as a numpy file
        output_filedir = os.path.join(mypath_output+ input_filedir[len(mypath):])
        Path(os.path.dirname(output_filedir)).mkdir(parents=True, exist_ok=True)
        np.save(output_filedir, M)
        
        
    print('Saving the ECT of directory in ', path, '...')


Now we have a saved dataset consisting of ECT matrices for the leaf outline dataset. We can train various models using these ECT matrices. Here, we show an example using a Convolutional Neural Network (CNN) to perform binary classification between the "Cotton" and "Ivy" classes.

# Train a CNN model on the ECT images

In [ ]:
import torch
import argparse
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm

This is the CNN model class we will use for classification, defined in `models.py`. Within this class, we specify the kernel size parameter. 
<div>
<img src="images/cnn_architecture.png" width="1000"/>
</div>

Also within `models.py`, the CNN class is defined using cylinder padding scheme for both of the convolution layers. To facilitate the illusion that the input images are cylindrical, we use padding defined by built in methods as part of the PyTorch package.
For each of the sides, where we want the left edge to be identified with the right edge of the image, we use circular padding which uses copies of columns from the right side to pad the left and copies of the left side to pad the right. On the top and bottom of each image, we pad with zero-valued pixels, which is a standard choice for computer vision tasks.
<div>
<img src="images/cylinder_and_zero_padding.jpeg" width="500"/>
</div>

In [ ]:
from models import CNN

These functions allow us to define datasets and load them into PyTorch. The dataset classes and data loaders can be adjusted in `dataloaders.py`.

In [ ]:
from dataloaders import create_datasets, create_data_loaders
from utils import save_model, save_plots, save_cf, SaveBestModel

Here we define a few model training parameters: the number of epochs to train the model and the learning rate. We also use a batch size of 4 samples, which is defined in `dataloaders.py`.

In [ ]:
NUM_EPOCHS = 50 # number of epochs to train the network for; type=int
LEARNING_RATE = 1e-3 # learning rate for training; type=float

In [ ]:
# device
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

### Load in the dataset and split into training and test datasets

Next, we build the training and test datasets as well as the dataloaders for each.

In [ ]:
train_dataset, test_dataset = create_datasets()

train_loader, test_loader = create_data_loaders(train_dataset, test_dataset)

Now, show a few samples from the training dataset. The labels for this example batch are printed as the title of each, where 0 indicates Ivy and 1 indicates Cotton'

In [ ]:
trainimages, trainlabels = next(iter(train_loader))
print(f'The tensor shape of each batch in the training dataset is \n[batch_size, num_channels, num_directions, num_thresholds]: {trainimages.shape}')

In [ ]:
plt.style.use('default')
fig, axes = plt.subplots(figsize=(10,4), ncols=4)
print('training images', trainimages.shape)
for i in range(4):
    ax = axes[i]
    ax.imshow(trainimages[i,0,:,:].T, cmap='gray')
    label = train_dataset.get_label(trainlabels[i])
    ax.set_title(label)
plt.show()

Next, we define two functions: `train()` and `validate()`. These define the training scheme and keep track of the loss during training (and validation) as well as the model accuracy.

In [ ]:
def train(model, train_loader, optimizer, lossfcn):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = lossfcn(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(train_loader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
# function for validation
def validate(model, valid_loader, lossfcn):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        outputs_list = []
        labels_list = []
        for i, data in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            counter += 1
            
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # forward pass
            outputs = model(image)
            outputs_list.append(outputs)
            labels_list.append(labels)
            # calculate the loss
            loss = lossfcn(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
        
    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(valid_loader.dataset))

    return epoch_loss, epoch_acc

Here, we specify the model to be used for training and print out a summary of the layers in the model.

In [ ]:
model = CNN(num_classes=train_dataset.num_classes, num_channels=trainimages.shape[1],input_resolution=(num_dirs,num_thresh)).to(device)
print(model)

We can also print out the total number of parameters and trainable parameters for the model for reference.

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

Next, we specifify the optimizer and loss function to be used for training. In this case we use the Adam optimizer built into PyTorch, which uses a gradient descent algorithm. We use a cross entropy loss function built into PyTorch, which computes the difference between two probability distributions and outputs a score (between 0 and 1, where 0 is a perfect score) summarizing the average difference between the predicted and actual values during training.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-3)
# loss function
lossfcn = nn.CrossEntropyLoss()
# initialize SaveBestModel class
save_best_model = SaveBestModel()

Now we actually train the model using our previously defined `train()` function! During training, we keep track of overall loss and accuracy values so that we can plot the behavior of the model throughout training.

In [ ]:
train_loss, valid_loss = [],[]
train_acc, valid_acc = [],[]

# begin training
for epoch in range(1,NUM_EPOCHS+1):
    print(f"[INFO]: Epoch {epoch} of {NUM_EPOCHS}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader, optimizer, lossfcn)
    valid_epoch_loss, valid_epoch_acc = validate(model, test_loader, lossfcn)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")

    # save the best model up to current epoch, if we have the least loss in the current epoch
    save_best_model(
        valid_epoch_loss, epoch, model, optimizer, lossfcn
    )
    print('-'*50)
    

Finally, we save the trained model weights and the loss and accuracy plots.

In [ ]:
save_model(NUM_EPOCHS, model, optimizer, lossfcn)
save_plots(train_acc, valid_acc, train_loss, valid_loss)

We can also visualize the model performance after training is complete thorugh a confusion matrix. We load in the best model and plot the confusion matrix.

In [ ]:
# model, valid_loader, lossfcn
model = CNN(num_classes=train_dataset.num_classes, num_channels=trainimages.shape[1],input_resolution=(num_dirs,num_thresh))

state_dict = torch.load('outputs/best_model.pth')['model_state_dict']
model.load_state_dict(state_dict)
model.eval()
model = model.to(device)
print('Using validation to compute confusion matrix')
valid_running_pred = []
valid_running_labels = []
counter = 0
with torch.no_grad():
    for i, data in tqdm(enumerate(test_loader), total=len(test_loader)):
        counter += 1
        
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        # forward pass
        outputs = model(image)
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)

        valid_running_pred.append(preds)
        valid_running_labels.append(labels)
    
# confusion matrix for the complete epoch
valid_running_pred = torch.cat(valid_running_pred)
valid_running_labels = torch.cat(valid_running_labels)
print('classes:',test_dataset.classes)
save_cf(valid_running_pred.cpu(),valid_running_labels.cpu(), test_dataset.classes)

In [ ]:
import numpy
numpy.round(numpy.linspace(60,144.4444,20))